# <center> Qual o impacto do gasto público no nº de mortes dos municípios brasileiros? </center>
## <center> Econometria I - EAE0324 </center>
### <center> Julho 2022 </center>
#### <center> Vitor Baldoino </center>

***

***Fontes de dados:***

- [Mortalidade](https://basedosdados.org/dataset/br-ms-sim?bdm_table=microdados)
- [Cadastro de empregos](https://basedosdados.org/dataset/br-me-caged?bdm_table=microdados_antigos)
- [Programa de Oportunidades](https://basedosdados.org/dataset/br-ipea-acesso-oportunidades?bdm_table=estatisticas_2019)
- [Despesas](https://basedosdados.org/dataset/br-me-siconfi?bdm_table=municipio_despesas_orcamentarias)
  - Também é possível fazer um trabalho só utilizando as despesas de um município.
- [Educação](https://basedosdados.org/dataset/br-inep-indicadores-educacionais?bdm_table=escola)
- [Tamanho da População](https://basedosdados.org/dataset/br-ibge-populacao?bdm_table=municipio)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_palette(sns.dark_palette("#2EA3C5", reverse=True, n_colors=8))
plt.style.use(['seaborn-paper'])
pd.set_option('display.max_columns', 500)

In [2]:
def filtrar_ano(df, ano):
    """Filtra um ano de uma DataFrame"""

    df = df.loc[df['ano'] == ano].reset_index()
    df = df.drop('index', axis='columns')
    return df

## Dados de Mortalidade

***

In [3]:
mortalidade = pd.read_csv('dados/trab_final/municipio_mortalidade_1996_2020.csv')
mortalidade = filtrar_ano(mortalidade, 2019)
mortalidade 

,ano,sigla_uf,id_municipio,numero_obitos
0,2019,AC,1200013,48
1,2019,AC,1200054,41
2,2019,AC,1200104,148
3,2019,AC,1200138,42
4,2019,AC,1200179,46
...,...,...,...,...
5565,2019,TO,1721208,139
5566,2019,TO,1721257,13
5567,2019,TO,1721307,7
5568,2019,TO,1722081,59


## Dados de Emprego

***

In [4]:
processamento_intermediario_emprego = """
empregos = None
erase = True

with pd.read_csv('dados/trab_final/microdados_antigos_ajustes_emprego.csv', chunksize=1000000, dtype=str,header=0, skiprows=[1, 35000000]) as reader:
    for chunk in reader:
        
        df = filtrar_ano(chunk, '2019')

        print(f"CHUNK: Formato = {chunk.shape}, Conteúdo = {chunk['ano'].unique()}")
        print(f"DF: Formato = {df.shape}, Conteúdo = {df['ano'].unique()}")
        
        if df.empty:
            pass
        else:
            if erase:
                df.to_csv('dados/trab_final/processamento_intermediario/microdados_antigos_emprego_2019.csv', mode='w', index=False)
                erase = False
            else:
                df.to_csv('dados/trab_final/processamento_intermediario/microdados_antigos_emprego_2019.csv', mode='a', header=False, index=False)
    df = None
"""

In [5]:
empregos = pd.read_csv('dados/trab_final/processamento_intermediario/microdados_antigos_emprego_2019.csv')
empregos = empregos[['id_municipio', 'tempo_emprego']]
empregos = empregos.groupby(by=['id_municipio'], as_index=False).mean()
empregos

c:\Users\vitor\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id_municipio,tempo_emprego
0,1100015,5.933333
1,1100023,4.656535
2,1100031,1.200000
3,1100049,10.061069
4,1100056,16.516129
...,...,...
4714,5222005,6.000000
4715,5222054,0.532468
4716,5222203,3.000000
4717,5222302,1.888889


## Estatísticas de Oportunidades

***

In [6]:
estatisticas_oportunidades = pd.read_csv('dados/trab_final/municipio_estatisticas_oportunidades_2019.csv')
estatisticas_oportunidades = estatisticas_oportunidades[['id_municipio', 'quantidade_pessoas', 'quantidade_pessoas_brancas', 'quantidade_pessoas_negras', 
                                                         'quantidade_pessoas_indigenas', 'quantidade_pessoas_amarelas', 'quantidade_estabelecimentos_ensino_infantil', 
                                                         'quantidade_estabelecimentos_ensino_fundamental', 'quantidade_estabelecimentos_ensino_medio']]

estatisticas_oportunidades = estatisticas_oportunidades.groupby(by=['id_municipio'], as_index=False).sum()
estatisticas_oportunidades
# Irei cruzar com outra base para pegar a UF de cada um desses municípios

,id_municipio,quantidade_pessoas,quantidade_pessoas_brancas,quantidade_pessoas_negras,quantidade_pessoas_indigenas,quantidade_pessoas_amarelas,quantidade_estabelecimentos_ensino_infantil,quantidade_estabelecimentos_ensino_fundamental,quantidade_estabelecimentos_ensino_medio
0,1302603,1780354,472241,1273131,3595,20122,261,556,133
1,1501402,1360366,374281,971199,2063,10821,110,283,116
2,2111300,984082,288423,680972,1578,10369,77,179,107
3,2304400,2405318,890723,1470664,2759,31970,209,367,169
4,2408102,790017,355447,421189,711,7626,85,171,60
5,2611606,1501982,627554,850616,3387,14120,231,313,109
6,2704302,921524,340459,560616,2189,10341,60,186,55
7,2927408,2598176,495206,2047881,7036,34927,266,516,183
8,3106200,2352062,1106265,1189832,2979,24100,186,379,147
9,3301702,828710,294140,523333,697,9057,155,221,81


## Dados Orçamentários

***

In [7]:
despesa_funcao = filtrar_ano(pd.read_csv('dados/trab_final/municipio_despesas_funcao.csv'), 2019)
despesa_funcao = despesa_funcao[['id_municipio', 'sigla_uf', 'conta_bd', 'valor']]

# Despesa total dos municipios
despesa_total = despesa_funcao.groupby(by=['id_municipio', 'sigla_uf'], as_index=False).sum().reset_index()

# Despesa contas selecionadas

seguranca = ['Segurança Pública', 'Policiamento', 'Defesa Civil', 'Demais Subfunções Segurança Pública',
             'Defesa Nacional', 'Demais Subfunções Defesa Nacional']

assistencia = ['Assistência Social', 'Assistência à Criança e ao Adolescente', 'Assistência Comunitária', 
               'Demais Subfunções Assistência Social', 'Assistência ao Idoso', 'Assistência ao Portador de Deficiência']

previdencia = ['Previdência Social', 'Previdência do Regime Estatutário', 'Previdência Básica', 'Demais Subfunções Previdência Social',
               'Previdência Especial', 'Previdência Complementar']

groups = despesa_funcao.loc[despesa_funcao['conta_bd'].isin(seguranca + assistencia + previdencia)].reset_index()
groups = groups.pivot_table(index=['id_municipio', 'sigla_uf'], columns='conta_bd', values='valor', aggfunc='sum').reset_index()


despesa = groups[['id_municipio', 'sigla_uf']].__deepcopy__()
despesa['seguranca'] = groups[seguranca].sum(axis=1)
despesa['assistencia'] =  groups[assistencia].sum(axis=1)
despesa['previdencia'] = groups[previdencia].sum(axis=1)
despesa['total'] = despesa_total['valor'].__deepcopy__()

despesa = despesa.drop(['sigla_uf'], axis='columns')
despesa

c:\Users\vitor\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


conta_bd,id_municipio,seguranca,assistencia,previdencia,total
0,1100015,0.00,3983858.82,0.00,5.441361e+08
1,1100023,10894446.66,24703658.91,53901636.99,2.168649e+09
2,1100031,0.00,7998366.38,0.00,2.316833e+08
3,1100049,1400.00,16363456.50,0.00,1.999057e+09
4,1100056,0.00,6878927.53,0.00,4.459262e+08
...,...,...,...,...,...
5532,5221908,0.00,4232203.92,9963813.54,5.877049e+08
5533,5222005,86894.58,13746819.24,27673947.60,1.726319e+08
5534,5222054,43158.48,8824517.16,21778772.34,1.317395e+08
5535,5222203,294462.80,3746449.56,11211697.80,1.573536e+08


## Dados de Educação

***

In [8]:
educacao = filtrar_ano(pd.read_csv('dados/trab_final/educacao_municipio.csv'), 2019)
educacao = educacao[['id_municipio', 'taxa_abandono_ef', 'taxa_reprovacao_ef', 'taxa_aprovacao_ef', 'taxa_abandono_em', 'taxa_reprovacao_em', 'taxa_aprovacao_em', 'atu_ei', 'atu_ef', 'atu_em']]

educacao = educacao.groupby(by=['id_municipio'], as_index=False).mean()
educacao

,id_municipio,taxa_abandono_ef,taxa_reprovacao_ef,taxa_aprovacao_ef,taxa_abandono_em,taxa_reprovacao_em,taxa_aprovacao_em,atu_ei,atu_ef,atu_em
0,1100015,1.583333,5.133333,93.283333,4.033333,6.633333,89.333333,15.000000,17.900000,13.633333
1,1100023,1.021429,6.414286,92.564286,6.846154,6.984615,86.169231,18.409091,24.807143,27.607692
2,1100031,0.181818,5.272727,94.545455,2.366667,2.533333,95.100000,21.000000,16.209091,13.800000
3,1100049,1.126667,3.286667,95.586667,2.821429,3.314286,93.864286,12.972727,20.246667,23.635714
4,1100056,0.300000,3.740000,95.960000,4.175000,2.750000,93.075000,14.000000,20.220000,19.600000
...,...,...,...,...,...,...,...,...,...,...
5565,5222005,0.038462,3.169231,96.792308,0.500000,3.600000,95.900000,17.854545,18.338462,28.000000
5566,5222054,0.225000,1.200000,98.575000,0.000000,1.900000,98.100000,22.900000,26.450000,23.200000
5567,5222203,0.981818,6.636364,92.381818,0.000000,0.000000,100.000000,22.100000,19.890909,28.700000
5568,5222302,0.000000,0.900000,99.100000,0.500000,2.033333,97.466667,13.633333,22.666667,22.100000


## Dados Populacionais

***

In [9]:
populacao = filtrar_ano(pd.read_csv('dados/trab_final/populacao_municipio.csv'), 2019)
populacao

,ano,sigla_uf,id_municipio,populacao
0,2019,RO,1100015,22945.0
1,2019,RO,1100023,107863.0
2,2019,RO,1100031,5312.0
3,2019,RO,1100049,85359.0
4,2019,RO,1100056,16323.0
...,...,...,...,...
5565,2019,GO,5222005,13863.0
5566,2019,GO,5222054,8743.0
5567,2019,GO,5222203,6171.0
5568,2019,GO,5222302,5821.0


## Base Consolidada

***

In [10]:

# Puxando base UF dos municípios da base de oportunidades para cruzar posteriormente
uf_oportunidades = pd.merge(estatisticas_oportunidades, populacao, how='left', on='id_municipio')
uf_oportunidades = uf_oportunidades.drop(['ano', 'populacao', 'id_municipio'], axis='columns')
uf_oportunidades = uf_oportunidades.groupby(by=['sigla_uf'], as_index=False).sum()
uf_oportunidades

,sigla_uf,quantidade_pessoas,quantidade_pessoas_brancas,quantidade_pessoas_negras,quantidade_pessoas_indigenas,quantidade_pessoas_amarelas,quantidade_estabelecimentos_ensino_infantil,quantidade_estabelecimentos_ensino_fundamental,quantidade_estabelecimentos_ensino_medio
0,AL,921524,340459,560616,2189,10341,60,186,55
1,AM,1780354,472241,1273131,3595,20122,261,556,133
2,BA,2598176,495206,2047881,7036,34927,266,516,183
3,CE,2405318,890723,1470664,2759,31970,209,367,169
4,DF,2537123,1062457,1405908,5426,39952,274,533,126
5,GO,1301930,622909,653153,1790,21551,258,257,83
6,MA,984082,288423,680972,1578,10369,77,179,107
7,MG,2352062,1106265,1189832,2979,24100,186,379,147
8,MS,783478,395725,365404,5279,13788,192,161,75
9,PA,1360366,374281,971199,2063,10821,110,283,116


In [11]:
base_final = pd.merge(mortalidade, empregos, how='left', on='id_municipio')
base_final = pd.merge(base_final, uf_oportunidades, how='left', on='sigla_uf')
base_final = pd.merge(base_final, despesa, how='left', on='id_municipio')
base_final = pd.merge(base_final, educacao, how='left', on='id_municipio')
base_final = pd.merge(base_final, populacao, how='left', on='id_municipio')
base_final = base_final.drop(['sigla_uf_y', 'ano_y'], axis='columns')

base_final = base_final.rename({'sigla_uf_x': 'sigla_uf', 'ano_x': 'ano', 'tempo_emprego': 'tempo_medio_emprego',
                                'quantidade_pessoas': 'qtd_pessoas_cidade_uf', 'quantidade_pessoas_brancas': 'qtd_brancos_cidade_uf',
                                'quantidade_pessoas_negras': 'qtd_negros_cidade_uf', 'quantidade_pessoas_indigenas': 'qtd_indigena_cidade_uf',
                                'quantidade_pessoas_amarelas': 'qtd_amarelos_cidade_uf', 'total': 'despesa_total'}, axis='columns')

base_final['log_populacao'] = np.log(base_final['populacao'])
base_final['log_obitos'] = np.log(base_final['numero_obitos'] + 1)
base_final['log_despesa'] = np.log(base_final['despesa_total'] + 1)
base_final['log_seguranca'] = np.log(base_final['seguranca'] + 1)
base_final['log_assistencia'] = np.log(base_final['assistencia'] + 1)
base_final['log_previdencia'] = np.log(base_final['previdencia'] + 1)

base_final['obitos_pc'] = base_final['numero_obitos'] / base_final['populacao']
base_final['despesa_pc'] = base_final['despesa_total'] / base_final['populacao']
base_final['seguranca_pc'] = base_final['seguranca'] / base_final['populacao']
base_final['assistencia_pc'] = base_final['assistencia'] / base_final['populacao']
base_final['previdencia_pc'] = base_final['previdencia'] / base_final['populacao']

base_final['maioria_nao_branca_cidade_uf'] = np.where(base_final[['qtd_negros_cidade_uf', 'qtd_indigena_cidade_uf', 'qtd_amarelos_cidade_uf']].sum(axis=1) > base_final['qtd_brancos_cidade_uf'], 1, 0)

norte = base_final['sigla_uf'].isin(['AM', 'AP', 'AC', 'PA', 'RO', 'RR', 'TO'])
nordeste = base_final['sigla_uf'].isin(['MA', 'PB', 'PE', 'PI', 'RN', 'SE', 'AL', 'BA', 'CE'])
sudeste = base_final['sigla_uf'].isin(['ES', 'MG', 'RJ', 'SP'])
sul = base_final['sigla_uf'].isin(['RS', 'SC', 'PR'])
centro = base_final['sigla_uf'].isin(['GO', 'MS', 'MT', 'DF'])

base_final['regiao'] = np.select(condlist=[norte, nordeste, sudeste, sul, centro], 
                                 choicelist=['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'])

base_final['norte'] = np.where(norte, 1, 0)
base_final['nordeste'] = np.where(nordeste, 1, 0)
base_final['sudeste'] = np.where(sudeste, 1, 0)
base_final['sul'] = np.where(sul, 1, 0)
base_final['centro'] = np.where(centro, 1, 0)

base_final['pop_quantile'] = pd.qcut(base_final['populacao'], q=[0, .25, .5, .75, 1.], labels=['A', 'B', 'C', 'D'])

base_final

,ano,sigla_uf,id_municipio,numero_obitos,tempo_medio_emprego,qtd_pessoas_cidade_uf,qtd_brancos_cidade_uf,qtd_negros_cidade_uf,qtd_indigena_cidade_uf,qtd_amarelos_cidade_uf,quantidade_estabelecimentos_ensino_infantil,quantidade_estabelecimentos_ensino_fundamental,quantidade_estabelecimentos_ensino_medio,seguranca,assistencia,previdencia,despesa_total,taxa_abandono_ef,taxa_reprovacao_ef,taxa_aprovacao_ef,taxa_abandono_em,taxa_reprovacao_em,taxa_aprovacao_em,atu_ei,atu_ef,atu_em,populacao,log_populacao,log_obitos,log_despesa,log_seguranca,log_assistencia,log_previdencia,obitos_pc,despesa_pc,seguranca_pc,assistencia_pc,previdencia_pc,maioria_nao_branca_cidade_uf,regiao,norte,nordeste,sudeste,sul,centro,pop_quantile
0,2019,AC,1200013,48,10.771429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,6253910.00,0.00,2.724452e+08,1.491667,3.441667,95.066667,13.233333,14.033333,72.733333,14.066667,26.658333,27.733333,15256.0,9.632728,3.891820,19.422948,0.000000,15.648718,0.000000,0.003146,17858.230874,0.000000,409.931175,0.000000,0,Norte,1,0,0,0,0,C
1,2019,AC,1200054,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,7852964.62,0.00,2.099017e+08,4.825000,7.825000,87.350000,6.333333,6.800000,86.866667,13.172727,18.491667,23.633333,7417.0,8.911530,3.737670,19.162150,0.000000,15.876402,0.000000,0.005528,28300.087438,0.000000,1058.779105,0.000000,0,Norte,1,0,0,0,0,B
2,2019,AC,1200104,148,4.959184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,12911431.04,871082.10,5.789614e+08,1.266667,2.583333,96.150000,3.566667,2.466667,93.966667,19.600000,24.158333,30.533333,26278.0,10.176487,5.003946,20.176746,0.000000,16.373624,13.677493,0.005632,22032.170992,0.000000,491.339944,33.148721,0,Norte,1,0,0,0,0,D
3,2019,AC,1200138,42,5.550000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.233333,2.375000,93.391667,5.766667,4.066667,90.166667,18.166667,20.725000,19.133333,10266.0,9.236593,3.761200,NaN,NaN,NaN,NaN,0.004091,NaN,NaN,NaN,NaN,0,Norte,1,0,0,0,0,B
4,2019,AC,1200179,46,5.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1340412.96,6125244.78,0.00,2.398449e+08,3.825000,9.000000,87.175000,10.066667,4.300000,85.633333,18.233333,24.033333,24.566667,11733.0,9.370161,3.850148,19.295503,14.108489,15.627929,0.000000,0.003921,20441.908374,114.242986,522.052738,0.000000,0,Norte,1,0,0,0,0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,2019,TO,1721208,139,2.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,20290248.47,0.00,9.250985e+09,2.135714,9.585714,88.278571,6.866667,17.766667,75.366667,13.545455,21.107143,25.700000,22870.0,10.037581,4.941642,22.947996,0.000000,16.825651,0.000000,0.006078,404503.050537,0.000000,887.199321,0.000000,0,Norte,1,0,0,0,0,C
5566,2019,TO,1721257,13,9.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,4837888.16,0.00,1.866738e+08,0.000000,3.350000,96.650000,0.000000,0.000000,100.000000,13.900000,21.200000,10.300000,1891.0,7.544861,2.639057,19.044873,0.000000,15.391989,0.000000,0.006875,98716.993866,0.000000,2558.375547,0.000000,0,Norte,1,0,0,0,0,A
5567,2019,TO,1721307,7,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35504.84,3954078.10,0.00,4.564499e+08,1.236364,9.909091,88.854545,3.100000,12.100000,84.800000,14.800000,16.827273,23.700000,2671.0,7.890208,2.079442,19.938989,10.477452,15.190258,0.000000,0.002621,170890.995915,13.292714,1480.373680,0.000000,0,Norte,1,0,0,0,0,A
5568,2019,TO,1722081,59,0.230769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40192.92,5891705.10,0.00,1.022676e+08,0.763636,2.590909,96.645455,1.400000,8.500000,90.100000,15.500000,20.472727,29.400000,11683.0,9.365890,4.094345,18.443103,10.601471,15.589056,0.000000,0.005050,8753.536026,3.440291,504.297278,0.000000,0,Norte,1,0,0,0,0,C


In [12]:
base_final.to_csv('dados/base_trab_final.csv', index=False)